In [1]:
import pandas as pd
import numpy as np

In [2]:
schedule = pd.read_csv('./schedule.csv', index_col=0)
ct = schedule['producer->compression_type'].to_list()
options = ['none', 'gzip', 'snappy', 'lz4']
for i, item in enumerate(ct):
    ct[i] = options.index(ct[i])
schedule['producer->compression_type'] = ct

In [3]:
pub_l = pd.read_csv('producer-latency.csv').sort_values(by=['index'])
pub_t = pd.read_csv('producer-throughput.csv').sort_values(by=['index'])
e2e_l = pd.read_csv('consumer-e2e_latency.csv', index_col=0).sort_values(by=['index'])
sub_l = pd.read_csv('consumer-latency.csv', index_col=0).sort_values(by=['index'])
sub_t = pd.read_csv('consumer-throughput.csv', index_col=0).sort_values(by=['index'])

In [4]:
states = pd.read_csv('states.csv', index_col=0).sort_values(by=['index'])
states = states.loc[:, (states != states.iloc[0]).any()]
states = states[~states.index.duplicated(keep='last')]
pub_l = pub_l[~pub_l.index.duplicated(keep='last')]
sub_l = sub_l[~sub_l.index.duplicated(keep='last')]
e2e_l = e2e_l[~e2e_l.index.duplicated(keep='last')]
pub_t = pub_t[~pub_t.index.duplicated(keep='last')]
sub_t = sub_t[~sub_t.index.duplicated(keep='last')]

In [5]:
valid_configs = np.intersect1d(states.index, sub_l.index)
valid_configs = np.intersect1d(valid_configs, pub_l.index)

In [6]:
states = states.loc[valid_configs]

In [7]:
xth = '90th'
states['pub-l'] = pub_l[xth]
states['sub-l'] = sub_l[xth]
states['e2e-l'] = e2e_l[xth]
states['pub-t'] = pub_t[xth]
states['sub-t'] = sub_t[xth]

In [8]:
states.to_csv('./processed/states.csv')

In [9]:
schedule

,producer->batch_size,producer->linger_ms,producer->buffer_memory,producer->compression_type,broker->num.network.threads,broker->num.io.threads,broker->queued.max.requests,consumer->fetch_wait_max_ms,consumer->max_partition_fetch_bytes,consumer->max_poll_records
index,,,,,,,,,,
0,111895,281,23194176,1,12,12,326,331,246735,300
1,798485,359,23751305,0,9,19,357,473,580452,356
2,948641,462,27848835,0,11,13,374,368,270330,144
3,265653,148,28736502,2,18,2,387,272,247698,138
4,928976,233,2979366,0,7,13,488,42,233551,446
...,...,...,...,...,...,...,...,...,...,...
1995,681161,492,26649574,1,4,15,417,85,645094,327
1996,556258,462,19205262,1,5,14,265,259,44963,236
1997,785391,363,1391358,0,13,18,59,481,305408,13


In [10]:
schedule = schedule.loc[valid_configs]
schedule.to_csv('./processed/schedule.csv')

In [11]:
raw_schedule = pd.read_csv('schedule.csv')
raw_index = raw_schedule['index'].to_list()
missed = []
for x in raw_index:
    if x not in schedule.index:
        missed.append(x)
missed_schedule = raw_schedule.iloc[missed]
missed_schedule.to_csv('missed.csv', index=None)